# How municipal median income affects the results of Finnish matricualtion examination

# 1. Abstract
In this study we gauge the effects of median income for a municipality on the results of the matriculation exam in different schools. We also form a modelin order to predict the finnish matriculation exam results for individual schools for the autumn term 2018.

# 2. Introduction and description of data

## Income data
Income data for different municipalities was obtained from the table incomes statistics, provided by Statistics Finland. Data for all finnish municipalities was available for the years 2005 - 2016. The data was obtained from the open data set provided by Statistics Finland on ther offical website [d1].

The data consists of metrics for each municipality. For each year there is one datum per municipality. The data contains the following variables of interest:
* $\texttt{year}$
* $\texttt{municipality}$, name of the municipality
* $\texttt{Number of income recipients}$
* $\texttt{Taxable income, median}$
* $\texttt{Municipal tax, mean}$
* Additional variables were not considered

The income data was joined with data linking each municipality name to a municipality code.

## Matriculation examination data
The Finnish matriculation examination is a naition-wide standardised test for upper secondary school. Students are given questions on different topics on multiple occasions in varying subjects, such as mother tongue, mathematics, physics, biology, languages and more. The exam is held twice a year (autumn and spring).

Data for the examination for all participating schools results was avaiable for the years 2012 - 2018 (spring). We used data for the years XX-YY. The data was obtained from the open data set provided by the Matriculation examinaton board on ther offical website [d2].

The data consists of a set of metrics for each school. For each year there is two datum per shool, one for each round. The data contains thefollowing variables of interest:
* $\texttt{round}$, examination round
* $\texttt{year}$, the year-part of the round
* $\texttt{season}$, the season part of the round (K=spring, S=autumn)
* $\texttt{school_id}$, school id number
* $\texttt{mean_ob}$, school mean of obligatory subject scores
* $\texttt{mean}$, school mean of all subjects
* $\texttt{n}$, number of students (for above metrics)
* $\texttt{mean_pkr}$, school mean of "advanced" subjects
* $\texttt{n_pkr}$, number of students (for above metric)
* Additional variables were not considered

The matriculation examination data was joined with data linking each school id to a municipality code.




# Hypothesis
Lorem ipsum.

# Methods
## Model
Dolor cit amet.

# Priors
Lorem ipsum.

In [1]:
import numpy as np
import pandas as pd
from data import Data
import config

In [3]:
df_income, df_results = Data.load()
df_all = pd.merge(df_results, df_income, on=["municipality_code", "year"], how='inner')

In [4]:
df_all.head()

,round,school_id,pass,hylatty,h_tai_k,graduated,mean_ob,mean,n,mean_pkr,n_pkr,municipality_code,year,season,municipality,number of income recipients,"taxable income, mean","taxable income, median"
0,2014K,1001,35,2.0,2.0,35,3.94,3.88,5.60,3.94,4.37,233,2014,K,Kauhava,14280,25046,21070
1,2014K,1269,57,2.0,2.0,56,4.48,4.40,5.07,4.52,3.86,233,2014,K,Kauhava,14280,25046,21070
2,2014S,1001,1,1.0,1.0,1,3.25,3.17,6.00,3.20,5.00,233,2014,S,Kauhava,14280,25046,21070
3,2014S,1269,1,1.0,1.0,1,3.25,3.00,5.00,3.00,4.00,233,2014,S,Kauhava,14280,25046,21070
4,2014K,1002,63,3.0,4.0,62,4.11,4.04,5.62,4.16,4.67,5,2014,K,Alajärvi,8372,22634,18729


# References

[1] Example (2018)  
Author, A. Examle publication

# Data sources
[d1] Data on taxablew income in Finland, for the years XX-YY  
     http://www.stat.fi/meta/til/tvt_en.html

[d2] Matriculation exam metrics for the schools in Finland, for the years XX-YY  
     https://www.ylioppilastutkinto.fi/tietopalvelut/tilastot/koulukohtaisia-tunnuslukuja
     
